In [ ]:
# =============================================================================
#
# --- QUBO Model Constructor for Portfolio Optimization ---
#
# Description:
#   This script builds a QUBO (Quadratic Unconstrained Binary Optimization)
#   model for a portfolio optimization problem. It loads bond data, defines
#   financial constraints (risk targeting and portfolio size), and constructs
#   the corresponding QUBO matrix `Q` and offset.
#
#   The resulting model is saved to text files, ready to be used by a
#   classical or quantum solver.
#
# Required Files:
#   - assets.csv: The raw data for all bonds.
#
# Main Outputs:
#   - qubo_matrix.txt: The generated QUBO matrix.
#   - asset_list.txt: The list of bond ISINs corresponding to the QUBO variables.
#   - qubo_offset.txt: The constant offset for the QUBO objective function.
#
# =============================================================================

import numpy as np
import pandas as pd
import random
import json

print("--- Building QUBO Model from Asset Data ---")

# --- 1. Load Data and Define Parameters ---
# --------------------------------------------
try:
    df = pd.read_csv('assets.csv')
    print("Successfully loaded 'assets.csv'.")
except FileNotFoundError:
    print("\nError: Could not find 'assets.csv'. Please make sure the file is in the same directory as this script.")
    exit()

# Define the size of the problem (number of assets to consider)
subset_size = 31
securities_df = df.head(subset_size).copy()
securities_df = securities_df.set_index('isin')
asset_list = securities_df.index.tolist()
num_assets = len(asset_list)
print(f"Building model for the first {num_assets} assets.")

# Define core optimization parameters
N = 10  # Target number of bonds in the final portfolio
characteristic_col = 'spreadDur' # The financial metric to target
betas = securities_df[characteristic_col]
lambda_penalty = 1.0  # Penalty coefficient for the size constraint

# Define risk buckets by grouping assets based on their credit quality
risk_bucket_col = 'security.elements.creditQualityBuckets'
buckets = securities_df.groupby(risk_bucket_col)
# The target for each bucket is the average of the characteristic for assets in that bucket
bucket_targets = {name: group[characteristic_col].mean() for name, group in buckets}

# --- 2. Manually Construct the QUBO Matrix (Q) and Offset ---
# ------------------------------------------------------------
# This section translates the financial rules into a mathematical QUBO form.
# ------------------------------------------------------------
Q = np.zeros((num_assets, num_assets))
offset = 0.0

# Add contributions from the Risk Bucket Hamiltonians (penalizes deviation from target)
print("Constructing QUBO matrix from risk bucket constraints...")
for name, group in buckets:
    K_target_lj = bucket_targets[name]
    bucket_isins = group.index.tolist()
    offset += K_target_lj**2
    for i in range(num_assets):
        if asset_list[i] not in bucket_isins: continue
        beta_i = betas[asset_list[i]]
        # Add linear term (diagonal of Q)
        Q[i, i] += beta_i**2 - 2 * beta_i * K_target_lj
        for j in range(i + 1, num_assets):
            if asset_list[j] not in bucket_isins: continue
            beta_j = betas[asset_list[j]]
            # Add quadratic term (off-diagonal of Q)
            Q[i, j] += 2 * beta_i * beta_j

# Add contributions from the Global Constraint Hamiltonian (penalizes incorrect portfolio size)
print("Adding global portfolio size constraint to QUBO matrix...")
offset += lambda_penalty * (N**2)
for i in range(num_assets):
    Q[i, i] += lambda_penalty * (1 - 2 * N)
    for j in range(i + 1, num_assets):
        Q[i, j] += lambda_penalty * 2

# Symmetrize the matrix, which is standard for QUBO representations
Q = (Q + Q.T) / 2

# --- 3. Save Artifacts to Text-Based Files ---
# ---------------------------------------------
# Save the complete model to robust, comma-delimited text files.
# ---------------------------------------------
np.savetxt('qubo_matrix.txt', Q, delimiter=',', fmt='%.18e')
with open('asset_list.txt', 'w') as f:
    for isin in asset_list:
        f.write(f"{isin}\n")
with open('qubo_offset.txt', 'w') as f:
    f.write(str(offset))

print("\nQUBO model files have been successfully generated:")
print("  - qubo_matrix.txt")
print("  - asset_list.txt")
print("  - qubo_offset.txt")

# Quantum Portfolio
# =============================================================================
#
# --- Quantum Portfolio Optimization using QAOA ---
#
# Description:
#   This script solves a portfolio optimization problem for 31 bonds using the
#   Quantum Approximate Optimization Algorithm (QAOA) implemented in Qiskit.
#   It loads a pre-computed QUBO model, converts it to an Ising Hamiltonian,
#   runs the QAOA on a high-performance simulator, and visualizes the
#   risk-return profile of the top resulting portfolios.
#
# Required Files:
#   - assets.csv: The raw data for all 31 bonds.
#   - qubo_matrix.txt: The QUBO matrix representing the optimization problem.
#   - asset_list.txt: A list of the bond ISINs in the correct order.
#   - qubo_offset.txt: The constant offset for the QUBO objective function.
#
# Main Outputs:
#   - Console output with the best portfolio found.
#   - qaoa_risk_return_plot.png: A plot showing the risk vs. return of top solutions.
#
# =============================================================================

import numpy as np
import pandas as pd
import qiskit
from qiskit_aer.primitives import Sampler
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import SparsePauliOp
import matplotlib.pyplot as plt

print("--- Quantum Portfolio Optimization with Risk-Return Visualization ---")

# --- 1. Load Problem Definition & Asset Data ---
# -----------------------------------------------
# Load the pre-computed QUBO model and the original asset data.
# The QUBO model was generated in a previous step to represent our
# specific portfolio constraints and objectives.
# -----------------------------------------------
try:
    Q = np.loadtxt('qubo_matrix.txt', delimiter=',')
    with open('asset_list.txt', 'r') as f:
        asset_list = [line.strip() for line in f.readlines()]
    with open('qubo_offset.txt', 'r') as f:
        offset = float(f.read())
    # Load original asset data for calculating risk/return for the final plot
    assets_df = pd.read_csv('assets.csv').set_index('isin')
    print("Successfully loaded QUBO model and asset data.")
except FileNotFoundError:
    print("Error: Make sure 'qubo_matrix.txt', 'asset_list.txt', and 'assets.csv' are in the same directory.")
    exit()

num_assets = len(asset_list)

# --- 2. Convert QUBO to Ising Hamiltonian ---
# --------------------------------------------
# Qiskit's QAOA works with Ising Hamiltonians, not QUBOs directly.
# This function converts the QUBO matrix (Q) into the corresponding
# Ising Hamiltonian (H) using the standard transformation x = (1 - Z)/2.
# --------------------------------------------
def get_ising_hamiltonian(Q_matrix):
    """Converts a QUBO matrix to an Ising Hamiltonian (SparsePauliOp)."""
    num_vars = Q_matrix.shape[0]
    pauli_list = []
    
    # Based on the standard QUBO to Ising conversion formulas:
    # J_ij = Q_ij / 4
    # h_i = sum_j(Q_ij)/4 + Q_ii/4
    J = Q_matrix / 4.0
    h = np.sum(Q_matrix, axis=0) / 4.0 + np.diag(Q_matrix) / 4.0
    ising_offset = (np.sum(Q_matrix) + np.sum(np.diag(Q_matrix))) / 4.0

    # Create the list of Pauli operators for the Hamiltonian
    for i in range(num_vars):
        if h[i] != 0:
            label = ['I'] * num_vars; label[i] = 'Z'
            pauli_list.append(("".join(label), -h[i]))
        for j in range(i + 1, num_vars):
            if J[i, j] != 0:
                label = ['I'] * num_vars; label[i] = 'Z'; label[j] = 'Z'
                pauli_list.append(("".join(label), J[i, j]))

    hamiltonian = SparsePauliOp.from_list(pauli_list)
    return hamiltonian, ising_offset

cost_hamiltonian, ising_offset = get_ising_hamiltonian(Q)
total_offset = offset + ising_offset
# print(f"Successfully converted QUBO to Ising Hamiltonian with {cost_hamiltonian.num_qubits} qubits.")

# --- 3. Set up and Run QAOA ---
# ------------------------------
# Configure the QAOA algorithm with a high-performance simulator, a classical
# optimizer, and the desired number of layers (reps).
# ------------------------------

# Use the Sampler primitive with the 'matrix_product_state' method.
# This is a highly efficient simulation method for QAOA-like circuits.
sampler = Sampler(backend_options={"method": "matrix_product_state"})

# Use COBYLA as the classical optimizer to tune the QAOA parameters.
optimizer = COBYLA(maxiter=100)

# reps=1 defines a single layer (p=1) QAOA ansatz.
qaoa = QAOA(sampler=sampler, optimizer=optimizer, reps=1)

print("\nRunning QAOA algorithm... (This may take a few minutes)")
result = qaoa.compute_minimum_eigenvalue(operator=cost_hamiltonian)
best_bitstring = result.best_measurement['bitstring']

# --- 4. Generate Risk-Return Visualization ---
# ---------------------------------------------
# To provide financial context to the quantum results, we calculate the
# risk and return for the top portfolios found by QAOA and plot them.
# ---------------------------------------------
print("\nGenerating risk-return plot for quantum results...")

# Get the probability distribution of all measured outcomes
eigenstate_probabilities = result.eigenstate.binary_probabilities()
# Sort the results by probability and select the top 10 for a clean plot
top_10_results = sorted(eigenstate_probabilities.items(), key=lambda item: item[1], reverse=True)[:10]

portfolio_risks = []
portfolio_returns = []

# Calculate risk and return for each of the top 10 portfolios
for bitstring, _ in top_10_results:
    selected_isins = [asset_list[i] for i, bit in enumerate(bitstring) if bit == '1']
    if not selected_isins: continue # Skip empty portfolios if they appear
    
    # Calculate average 'oas' as return and 'spreadDur' as risk
    portfolio_data = assets_df.loc[selected_isins]
    portfolio_returns.append(portfolio_data['oas'].mean())
    portfolio_risks.append(portfolio_data['spreadDur'].mean())

# Identify the risk/return of the single best solution for highlighting
best_portfolio_data = assets_df.loc[[asset_list[i] for i, bit in enumerate(best_bitstring) if bit == '1']]
best_risk = best_portfolio_data['spreadDur'].mean()
best_return = best_portfolio_data['oas'].mean()

# Create the scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(portfolio_risks, portfolio_returns, c='blue', label='Other Top QAOA Solutions')
plt.scatter(best_risk, best_return, c='red', marker='*', s=200, label='Best QAOA Solution', zorder=5)

plt.xlabel('Portfolio Risk (Average Spread Duration)')
plt.ylabel('Portfolio Return (Average OAS)')
plt.title('Risk vs. Return Profile of Top Portfolios from QAOA')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("qaoa_risk_return_plot.png")
print("Saved risk-return plot to 'qaoa_risk_return_plot.png'")


# --- 5. Decode and Display Final Text Results ---
# ------------------------------------------------
# Print the final, human-readable portfolio solution.
# ------------------------------------------------
optimal_value = result.best_measurement['value'] + total_offset
selected_assets = [asset_list[i] for i, bit in enumerate(best_bitstring) if bit == '1']

print("\n--- QAOA Final Results ---")
print(f"Optimal objective value found: {optimal_value:.4f}")
print(f"Most likely solution bitstring: {best_bitstring}")
print(f"\nOptimal portfolio found with {len(selected_assets)} bonds:")
print("------------------------------------------")
for isin in selected_assets:
    print(f"  - {isin}")
print("------------------------------------------")
